- Grab new songs for genre from spotify -> feasible?
- Run popularity prediction
- Return sample

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('SpotifyFeatures.csv')
print(df.columns)
df.describe()

Index(['genre', 'artist_name', 'track_name', 'track_id', 'popularity',
       'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence'],
      dtype='object')


,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
count,232725.000000,232725.000000,232725.000000,2.327250e+05,232725.000000,232725.000000,232725.000000,232725.000000,232725.000000,232725.000000,232725.000000
mean,41.127502,0.368560,0.554364,2.351223e+05,0.570958,0.148301,0.215009,-9.569885,0.120765,117.666585,0.454917
std,18.189948,0.354768,0.185608,1.189359e+05,0.263456,0.302768,0.198273,5.998204,0.185518,30.898907,0.260065
min,0.000000,0.000000,0.056900,1.538700e+04,0.000020,0.000000,0.009670,-52.457000,0.022200,30.379000,0.000000
25%,29.000000,0.037600,0.435000,1.828570e+05,0.385000,0.000000,0.097400,-11.771000,0.036700,92.959000,0.237000
50%,43.000000,0.232000,0.571000,2.204270e+05,0.605000,0.000044,0.128000,-7.762000,0.050100,115.778000,0.444000
75%,55.000000,0.722000,0.692000,2.657680e+05,0.787000,0.035800,0.264000,-5.501000,0.105000,139.054000,0.660000
max,100.000000,0.996000,0.989000,5.552917e+06,0.999000,0.999000,1.000000,3.744000,0.967000,242.903000,1.000000


In [3]:
pd.isnull(df).sum()

genre               0
artist_name         0
track_name          1
track_id            0
popularity          0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
speechiness         0
tempo               0
time_signature      0
valence             0
dtype: int64

In [4]:
df.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


In [5]:
df["key_num"] = df["key"].astype('category').cat.codes
df["mode_num"] = df["mode"].astype('category').cat.codes
df["time_signature_num"] = df["time_signature"].astype('category').cat.codes

In [6]:
df.head(10)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,...,liveness,loudness,mode,speechiness,tempo,time_signature,valence,key_num,mode_num,time_signature_num
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.61100,0.389,99373,0.9100,0.00000,...,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814,4,0,3
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.24600,0.590,137373,0.7370,0.00000,...,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816,9,1,3
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.95200,0.663,170267,0.1310,0.00000,...,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368,3,1,4
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.70300,0.240,152427,0.3260,0.00000,...,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227,4,0,3
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.95000,0.331,82625,0.2250,0.12300,...,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390,8,0,3
5,Movie,Henri Salvador,Le petit souper aux chandelles,0Mf1jKa8eNAf1a4PwTbizj,0,0.74900,0.578,160627,0.0948,0.00000,...,0.1070,-14.970,Major,0.1430,87.479,4/4,0.358,4,0,3
6,Movie,Martin & les fées,"Premières recherches (par Paul Ventimila, Lori...",0NUiKYRd6jt1LKMYGkUdnZ,2,0.34400,0.703,212293,0.2700,0.00000,...,0.1050,-12.675,Major,0.9530,82.873,4/4,0.533,4,0,3
7,Movie,Laura Mayne,Let Me Let Go,0PbIF9YVD505GutwotpB5C,15,0.93900,0.416,240067,0.2690,0.00000,...,0.1130,-8.949,Major,0.0286,96.827,4/4,0.274,9,0,3
8,Movie,Chorus,Helka,0ST6uPfvaPpJLtQwhE6KfC,0,0.00104,0.734,226200,0.4810,0.00086,...,0.0765,-7.725,Major,0.0460,125.080,4/4,0.765,3,0,3
9,Movie,Le Club des Juniors,Les bisous des bisounours,0VSqZ3KStsjcfERGdcWpFO,10,0.31900,0.598,152694,0.7050,0.00125,...,0.3490,-7.790,Major,0.0281,137.496,4/4,0.718,10,0,3


In [7]:
# Deal with NaN
df = df.dropna()
# Deal with duplicates

# Deal with data normalization

In [8]:
features = ["acousticness", "danceability", "duration_ms", "energy", "instrumentalness", "liveness", "loudness", "speechiness", "tempo", "valence", "key_num", "mode_num", "time_signature_num"]

In [9]:
x = df[features]
y = df["popularity"]

In [10]:
#normalize x values
x = (x - x.mean()) / x.std()

In [11]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [12]:
len(x_train)

186179

In [13]:
x_train = x_train[:100000]
y_train = y_train[:100000]

x_test = x_test[:100000]
y_test = y_test[:100000]

In [15]:
# do a linear regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import tensorflow as tf
from tensorflow import keras

model = LinearRegression()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print(mean_squared_error(y_test, y_pred))

# do a random forest

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print(mean_squared_error(y_test, y_pred))

# do a neural network

model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=[len(features)]),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')

model.fit(x_train, y_train, epochs=100)

y_pred = model.predict(x_test)
model.save('spotify_model_1.h5')

mean_squared_error(y_test, y_pred)

254.25024999771114
167.71030436039274
Epoch 1/100


c:\Users\simon\OneDrive\LMU\Master\1. Semester\PML\SpotifyPopularityPrediction\venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3125/3125 ━━━━━━━━━━━━━━━━━━━━ 3s 533us/step - loss: 372.8224
Epoch 2/100
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 526us/step - loss: 224.5671
Epoch 3/100
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 529us/step - loss: 218.0912
Epoch 4/100
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 532us/step - loss: 214.9465
Epoch 5/100
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 547us/step - loss: 215.0305
Epoch 6/100
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 533us/step - loss: 211.8401
Epoch 7/100
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 531us/step - loss: 209.5025
Epoch 8/100
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 533us/step - loss: 210.4367
Epoch 9/100
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 555us/step - loss: 208.7154
Epoch 10/100
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 554us/step - loss: 210.6101
Epoch 11/100
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 543us/step - loss: 209.9847
Epoch 12/100
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 542us/step - loss: 206.9117
Epoch 13/100
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 542us/step - loss: 206.1474
Epoch 14/100
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s

224.68637524763275

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# client_id = ''
# client_secret = ''

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id, client_secret))

# Get a list of new releases
new_releases = spotify.new_releases('US', limit=50)
albums = new_releases['albums']['items']

for album in albums:
   # Get the artists on the album
   artists = album['artists']
   for artist in artists:
      # Get the genres for the artist
      artist_info = spotify.artist(artist['id'])
      genres = artist_info['genres']

      # If the desired genre is in the artist's genres, print their name
      if 'pop' in genres:
         print(artist['name'])

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = 'your client id'
client_secret = 'your client secret'
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id, client_secret))

# specify the preferred genre
preferred_genre = 'pop'

# Get a list of new releases
new_releases = spotify.new_releases('US', limit=50)
albums = new_releases['albums']['items']
track_list = []

# Go through each album
for album in albums:
    album_id = album['id']
    tracks = spotify.album_tracks(album_id)
    for track in tracks['items']:
        artists = track['artists']
        for artist in artists:
            artist_info = spotify.artist(artist['id'])
            genres = artist_info['genres']

            # If the preferred genre is in the artist genres, get the features of the song
            if preferred_genre in genres:
                features = spotify.audio_features([track['id']])
                track_list.append({'name': track['name'], 'features': features})

            # If the track list is more than 10, return it
            if len(track_list) > 10:
                break
        if len(track) > 10:
            break
    if len(track_list) > 10:
            break

for track in track_list:
     print(track)